# Introduction to network communication with Scapy

In this short tutorial, we will use Scapy library to generate simple text and audio traffic and send it across the network.

## Preparation

To fully understand the topic at hand, please make a PCAP file of your phone call to the test PBX and analyze it.

For this activity we will also need an audio recording in the format of signed linear 16 bit PCM encoded samples in WAV container.

## Imports

In [ ]:
import scapy.all as sa
import numpy as np
import time
from scipy.io import wavfile

## Constants

In [ ]:
rng = np.random.default_rng()
SRC_IP = "127.0.0.1" # get this address by calling 'ip a' command
DST_IP = "127.0.0.1" # get this from your colleague
SRC_PORT = 12345
DST_PORT = 54321

## Simple text exchange

This snippet creates an ICMP packet carrying a simple text message. First we start with a carrying IP packet that will take care of the delivery across the network.

In [ ]:
ip = sa.IP(src=SRC_IP, dst=DST_IP)

This is how the packet is represented as a Python string.

In [ ]:
ip

Now, let's look how it would appear BEFORE finalization to send it.

In [ ]:
ip.show()

And how it will look like 'on the wire'.

In [ ]:
ip.show2()

Now, we will add a new layer on top of the IP packet. In our case, it is going to be an [ICMP](https://en.wikipedia.org/wiki/Internet_Control_Message_Protocol) packet.

In [ ]:
icmp = sa.ICMP(type=8)

In [ ]:
icmp

And lastly, we will add a message.

In [ ]:
msg = "Hello world"

Now, let's tie it all together.

In [ ]:
packet = ip / icmp / msg

In [ ]:
packet.show2()

In [ ]:
sa.hexdump(packet)

### Question
Where do you know this format from and what does it mean?

### Sender

In [ ]:
sa.send(packet)

### Receiver

In [ ]:
def pkt_callback(pkt):
    if pkt["ICMP"].type == 8:
        print(pkt["Raw"].load, end="")

sa.sniff(iface="lo", prn=pkt_callback, filter="icmp", store=0)

## Task
- Modify the code so that it uses UDP instead and sends message letter by letter. Explain differences.
- Update your sender to dynamically adjust send time of a packet and introduce a packet loss

## Audio transmission

Following snippet sends audio encapsulated in RTP packet in the form of G.711 encoded samples.

In [ ]:
from lib.core import linear_to_alaw, linear_to_ulaw

def send_audio_as_rtp(encoding='alaw'):
  """
  Send audio as RTP packets with specified encoding
  encoding: 'ulaw' for μ-law or 'alaw' for A-law
  """
  
  # Select encoding parameters
  if encoding.lower() == 'ulaw':
      payload_type = 0  # PCMU
      converter = linear_to_ulaw
  elif encoding.lower() == 'alaw':
      payload_type = 8  # PCMA
      converter = linear_to_alaw
  else:
      raise ValueError("Encoding must be 'ulaw' or 'alaw'")
  
  # Bind UDP to RTP for the destination port
  sa.bind_layers(sa.UDP, sa.RTP, dport=DST_PORT)
  
  try:
      # Read WAV file using scipy
      sample_rate, samples = wavfile.read('input.wav')
      
      # Check audio format
      if sample_rate != 8000:
          print("Error: Audio must be 8000 Hz")
          return
          
      if samples.dtype != np.int16:
          print("Error: Audio must be 16-bit PCM")
          return
      
      
      # Calculate packet duration (20ms = 160 samples for 8kHz)
      samples_per_packet = 160
      
      sequence_number = 0
      timestamp = 0
      
      # Process audio in chunks
      for i in range(0, len(samples), samples_per_packet):
          # Get chunk of samples
          chunk = samples[i:i + samples_per_packet]
          
          # Pad last chunk if necessary
          if len(chunk) < samples_per_packet:
              chunk = np.pad(chunk, (0, samples_per_packet - len(chunk)))
          
          # Convert to 8-bit encoded format
          encoded_samples = converter(chunk)
          
          # Create RTP packet using Scapy's RTP layer
          packet = sa.IP(dst=DST_IP)/sa.UDP(sport=SRC_PORT, dport=DST_PORT)/sa.RTP(
              version=2,
              padding=0,
              extension=0,
              marker=0,
              payload_type=payload_type,
              sequence=sequence_number,
              timestamp=timestamp,
              sourcesync=0x12345678
          )/sa.Raw(load=bytes(encoded_samples))
          
          # Send packet
          sa.send(packet, verbose=False)
          
          # Update sequence number and timestamp
          sequence_number = (sequence_number + 1) & 0xFFFF
          timestamp = (timestamp + samples_per_packet) & 0xFFFFFFFF
          
          # Wait to maintain real-time playback (20ms)
          time.sleep(0.01)
          
  except Exception as e:
      print(f"Error processing audio: {e}")

In [ ]:
send_audio_as_rtp()

## Task
- Use tcpdump/wireshark to sniff and listen to audio.
- Based on information gained, assess the effectiveness and efficiecy of the code.
- Introduce variable delays and/or packet loss.
- Evaluate transmitted audio.
- Visualize losses and/or delays.

When done, continue with the activity according to the guide specified in the README.md.